Make BGP-LLaMa and Mobile LLaMa web services.

With the web service, develop the real user cases for the network
operation and administrators.
For example, I think the real-time monitoring function and the
generation report in the natural language.
- IP prefix comparison from different regional bgp sources
- Specific AS report: AS18298 (CNU) BGP report for last 5 minutes
- Hourly/daily BGP monitoring summarization/report with plots
  * We can directly use 5-minute near real-time data of  bgp routing
tables and update messages.
- BGP anomaly detection reports

In [ ]:
import pybgpstream
stream = pybgpstream.BGPStream(
    # accessing ris-live
    project="ris-live",
    # filter to show only stream from rrc00
    filter="collector rrc00",
)

for elem in stream:
    print(elem)

In [ ]:
import pybgpstream
stream = pybgpstream.BGPStream(
    # accessing routeview-stream
    project="routeviews-stream",
    # filter to show only stream from amsix bmp stream
    filter="router amsix",
)

for elem in stream:
    print(elem)

In [ ]:
import pybgpstream

# Create a BGPStream instance
stream = pybgpstream.BGPStream(
    # Specify data source, e.g., 'routeviews', 'ris'
    project="ris",
    # Use 'record_type' as 'updates' to get real-time BGP update messages
    record_type="updates",
    # Optionally filter data (e.g., by specific prefix, ASN, etc.)
    # filter="prefix 1.1.1.0/24"
)

# Start the live stream
for rec in stream.records():
    # Process each record
    print(f"Time: {rec.time}")
    for elem in rec:
        # Print out details of each BGP element
        print(f"Peer ASN: {elem.peer_asn}, Peer IP: {elem.peer_address}, Type: {elem.type}")
        if elem.type in ('A', 'W'):  # Announcements and Withdrawals
            print(f"Prefix: {elem.fields['prefix']}, AS Path: {elem.fields.get('as-path', 'N/A')}")
        print("-" * 40)